Introduction

In [7]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

Data preparation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Paths
data_dir = '../dataset/asl_alphabet_train'

# Data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # 20% for validation
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

# Training generator
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(200, 200),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

# Validation generator
val_generator = val_datagen.flow_from_directory(
    data_dir,
    target_size=(200, 200),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)


Found 69600 images belonging to 29 classes.
Found 17400 images belonging to 29 classes.


Building model

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras import Input

model = Sequential([
    # Block 1
    Input(shape=(200, 200, 3)),  # Explicit Input layer
    Conv2D(32, (5, 5), activation='relu'),
    MaxPooling2D(pool_size=(2, 2), strides=2),

    # Block 2
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2), strides=2),

    # Block 3
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2), strides=2),

    # Flatten and Dense layers
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),

    # Output layer
    Dense(29, activation='softmax')
])

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 196, 196, 32)   │         2,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 98, 98, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 96, 96, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 48, 48, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 46, 46, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 23, 23, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 67712)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │    17,334,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 29)             │         7,453 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,436,765 (66.52 MB)

 Trainable params: 17,436,765 (66.52 MB)

 Non-trainable params: 0 (0.00 B)

Compile the model

In [ ]:
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

Define Early Stopping Callback

In [ ]:
early_stopping = EarlyStopping(
    monitor='val_accuracy',  # Monitor validation accuracy
    patience=3,              # Stop after 3 epochs without improvement
    restore_best_weights=True  # Keep the best model weights
)

Training the model:

1. Fit the model

Initial run:

In [11]:
# Fit the model to the training data with validation and early stopping
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30,
    batch_size=32,
    callbacks=[early_stopping],
    verbose=1
)


Epoch 1/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 0s 9s/step - accuracy: 0.2288 - loss: 2.6480

c:\Users\adith\Desktop\Projects\DM\ASL-CNN-Project\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2175/2175 ━━━━━━━━━━━━━━━━━━━━ 19101s 9s/step - accuracy: 0.2289 - loss: 2.6477 - val_accuracy: 0.5495 - val_loss: 1.3226
Epoch 2/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 762s 350ms/step - accuracy: 0.6369 - loss: 1.0819 - val_accuracy: 0.6314 - val_loss: 1.0694
Epoch 3/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 760s 350ms/step - accuracy: 0.7580 - loss: 0.7090 - val_accuracy: 0.7266 - val_loss: 0.8369
Epoch 4/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 1251s 575ms/step - accuracy: 0.8114 - loss: 0.5464 - val_accuracy: 0.7637 - val_loss: 0.7704
Epoch 5/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 3264s 2s/step - accuracy: 0.8499 - loss: 0.4441 - val_accuracy: 0.7723 - val_loss: 0.7978
Epoch 6/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 3729s 2s/step - accuracy: 0.8664 - loss: 0.3907 - val_accuracy: 0.7832 - val_loss: 0.7248
Epoch 7/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 3675s 2s/step - accuracy: 0.8866 - loss: 0.3384 - val_accuracy: 0.7861 - val_loss: 0.7548
Epoch 8/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 3681s 2s/step - accuracy: 0.8972 - lo